In [40]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt

In [41]:
house_2018 = pd.read_csv('house_polls_2018_inprogress.csv')

In [42]:
#Move from one row per candidate to one row per poll
#What's the last poll we've been working on?
oldno = -10
#What's the index of that last poll?
masterindex = 0
for index, pollno in enumerate(house_2018['poll_id']):
    #If we've seen this poll_id before, this row deals with a second, third, etc candidate:
    if oldno == pollno:
        #If we haven't set stats for candidate 2 yet:
        if np.isnan(house_2018['cand2_pct'][masterindex]):
            #Set them equal to this row
            house_2018['cand2_pct'][masterindex] = house_2018['cand1_pct'][index]
            house_2018['cand2_name'][masterindex] = house_2018['cand1_name'][index]
        #If we're still on this poll, there could be a third candidate, but their name isn't tracked
        elif np.isnan(house_2018['cand3_pct'][masterindex]):
            house_2018['cand3_pct'][masterindex] = house_2018['cand1_pct'][index]
        #Some polls track even more candidates, but our model doesn't, so just don't do anything 
        #if there's even more rows from this poll
    else:
        #This is a new poll, so update the reference information and keep going
        oldno = pollno
        masterindex = index

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [43]:
#Remove all superfluous rows
house_2018 = house_2018[~np.isnan(house_2018['cand2_pct'])]

In [44]:
house_2018['margin_poll'] = (house_2018['cand1_pct'] - house_2018['cand2_pct'])

In [45]:
#Credit to Mike Shultz for creating this
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

In [46]:
state_key = {state: abbr for abbr, state in states.items()}

In [47]:
house_results = pd.read_csv('house_results_2018.csv')

In [48]:
house_results = house_results.set_index('District').transpose()

In [49]:
#Add in results data and data analysis
#Still need to rework the rightcall code
house_2018['location'] = house_2018['state']
house_2018['district'] = house_2018['state']
house_2018 = house_2018.reset_index()
for index in range(len(house_2018)):
    state = house_2018['state'][index]
    house_2018['location'][index] = state_key[state]
    house_2018['district'][index] = (str(house_2018['location'][index])+'-'+str(house_2018['seat_number'][index]))
    abbr = house_2018['district'][index]
    house_2018['cand1_actual'][index] = house_results[abbr][0]
    house_2018['cand2_actual'][index] = house_results[abbr][1]
    house_2018['margin_actual'] = house_2018['cand1_actual'] - house_2018['cand2_actual']
    house_2018['margin_poll'] = house_2018['cand1_pct'] - house_2018['cand2_pct']
    house_2018['bias'] = house_2018['margin_poll'] - house_2018['margin_actual']
    house_2018['error'] = abs(house_2018['bias'])
    #if ((house_2018['cand1_pct'] > house_2018['cand2_pct']) and (house_2018['cand1_actual'] > house_2018['cand2_actual'])) or ((house_2018['cand1_pct'] < house_2018['cand2_pct']) and (house_2018['cand1_actual'] < house_2018['cand2_actual'])):
        #house_2018['rightcall'] = 1
    #else:
         #house_2018['rightcall'] = 0

C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\marcu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\marcu\

In [50]:
house_2018

,index,poll_id,cycle,state,pollster,sample_size,population_full,methodology,office_type,seat_number,...,margin_poll,election_date,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,location,district
0,0,56821,2018,Florida,St. Pete Polls,1194.0,lv,Automated Phone,U.S. House,15,...,-1.9,11/6/2018,47.0,53.0,-6.0,4.1,4.1,NaN,FL,FL-15
1,2,56760,2018,South Dakota,Change Research,851.0,lv,Online,U.S. House,1,...,-10.0,11/6/2018,36.0,60.3,-24.3,14.3,14.3,NaN,SD,SD-1
2,6,56761,2018,Wyoming,Change Research,858.0,lv,Online,U.S. House,1,...,-27.0,11/6/2018,29.8,63.7,-33.9,6.9,6.9,NaN,WY,WY-1
3,10,56762,2018,Montana,Change Research,879.0,lv,Online,U.S. House,1,...,-8.0,11/6/2018,46.2,50.9,-4.7,3.3,-3.3,NaN,MT,MT-1
4,13,56788,2018,Georgia,Change Research,395.0,lv,Online,U.S. House,6,...,6.0,11/6/2018,50.5,49.5,1.0,5.0,5.0,NaN,GA,GA-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,1721,52749,2018,Iowa,Public Policy Polling,1093.0,v,NaN,U.S. House,1,...,2.0,11/6/2018,51.0,45.9,5.1,3.1,-3.1,NaN,IA,IA-1
531,1723,52748,2018,Colorado,Public Policy Polling,742.0,v,NaN,U.S. House,6,...,-7.0,11/6/2018,54.1,45.9,8.2,15.2,-15.2,NaN,CO,CO-6
532,1725,53605,2018,Minnesota,Public Policy Polling,732.0,v,Automated Phone,U.S. House,2,...,-1.0,11/6/2018,52.8,47.2,5.6,6.6,-6.6,NaN,MN,MN-2
533,1727,52675,2018,California,Tulchin Research,401.0,lv,NaN,U.S. House,48,...,-4.0,11/6/2018,53.6,46.4,7.2,11.2,-11.2,NaN,CA,CA-48


In [52]:
houseTest = house_2018.groupby('methodology').mean()

In [53]:
houseTest['error']

methodology
Automated Phone      8.472500
IVR                  9.300000
IVR/Live Phone       5.111765
IVR/Online           5.845000
Live Phone           6.450885
Live Phone/Online    2.075000
Online               5.346875
Name: error, dtype: float64